In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from google.colab import files

In [ ]:
# Data Loading and Processing
def load_and_process_data(file_path):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File {file_path} not found")

        df = pd.read_excel(file_path, sheet_name='Worksheet')
        print(f"Raw data loaded: {df.shape[0]} rows, {df.shape[1]} columns")

        required_cols = {'bts_id', 'bts_status', 'latitude', 'longitude',
                        'vendor_name', 'tower_type', 'site_category', 'bts_area'}
        missing = required_cols - set(df.columns)
        if missing:
            raise ValueError(f"Missing columns: {missing}")

        df = df[df['bts_status'].str.strip().str.lower() == 'up']
        df = df.dropna(subset=['latitude', 'longitude'])
        df = df.drop_duplicates(subset=['bts_id'])
        print(f"Active BTS after filtering: {len(df)}")

        coords = df[['latitude', 'longitude']].values
        n_neighbors = min(6, len(coords))
        nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(coords)
        _, indices = nbrs.kneighbors(coords)

        handover_targets = []
        for i in range(len(df)):
            neighbors = indices[i][1:]
            distances = [geodesic(coords[i], coords[n]).km for n in neighbors]
            weights = 1 / (np.array(distances) + 0.1)
            weights /= weights.sum()
            handover_targets.append(np.random.choice(neighbors, p=weights))

        df['target'] = handover_targets
        return df.reset_index(drop=True)

    except Exception as e:
        print(f"Data processing failed: {str(e)}")
        raise

In [ ]:
class FeatureEngineer:
    def __init__(self):
        self.scaler = StandardScaler()
        self.encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    def fit_transform(self, df):
        num_feats = self.scaler.fit_transform(df[['latitude', 'longitude']])
        cat_feats = self.encoder.fit_transform(df[['vendor_name', 'tower_type', 'site_category', 'bts_area']])
        return np.hstack([num_feats, cat_feats])

    def transform(self, df):
        num_feats = self.scaler.transform(df[['latitude', 'longitude']])
        cat_feats = self.encoder.transform(df[['vendor_name', 'tower_type', 'site_category', 'bts_area']])
        return np.hstack([num_feats, cat_feats])

In [ ]:
# Graph Construction
def build_adjacency_matrix(coords, threshold_km=8):
    n = len(coords)
    adj = np.zeros((n, n), dtype='float32')

    for i in range(n):
        for j in range(n):
            if i != j:
                dist = geodesic(coords[i], coords[j]).km
                if dist < threshold_km:
                    adj[i][j] = 1 / (1 + dist)

    row_sums = adj.sum(axis=1) + 1e-10
    D = np.diag(1 / np.sqrt(row_sums))
    return D @ adj @ D

In [ ]:
# GCN Model
class HandoverPredictor(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.classifier = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        features, adj = inputs
        x = self.dense1(features)
        x = tf.matmul(adj, x)
        x = self.dense2(x)
        x = tf.matmul(adj, x)
        return self.classifier(x)


In [ ]:
def train_model(data_path, threshold_km=8):
    try:
        df = load_and_process_data(data_path)
        coords = df[['latitude', 'longitude']].values

        engineer = FeatureEngineer()
        X = engineer.fit_transform(df)
        adj = build_adjacency_matrix(coords, threshold_km)

        X_tensor = tf.constant(X, dtype=tf.float32)
        adj_tensor = tf.constant(adj, dtype=tf.float32)
        y_tensor = tf.constant(df['target'], dtype=tf.int32)

        train_mask = np.random.rand(len(df)) < 0.8
        val_mask = ~train_mask

        model = HandoverPredictor(X.shape[1], len(df))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        print("\nStarting training...")
        history = model.fit(
            x=(X_tensor, adj_tensor),
            y=y_tensor,
            sample_weight=train_mask,
            epochs=100,
            batch_size=len(df),
            validation_data=((X_tensor, adj_tensor), y_tensor, val_mask),
            callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=10),
                tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
            ]
        )

        return model, engineer, adj, df

    except Exception as e:
        print(f"\nTraining failed: {str(e)}")
        return None, None, None, None

In [ ]:
def predict_handover(model, engineer, adj, df, current_bts_id, user_location):
    """Generates recommendations with full graph processing"""
    try:
        current_idx = df.index[df['bts_id'] == current_bts_id][0]

        # Get predictions for ALL nodes
        features = engineer.transform(df)

        # Add batch dimension and predict
        probs = model.predict(
            (features[np.newaxis], adj[np.newaxis]),  # Shape: [1,190,14] and [1,190,190]
            verbose=0
        )[0]  # Output shape: [190,190]

        # Extract probabilities for current node
        current_probs = probs[current_idx]

        # Calculate real-time distances
        coords = df[['latitude', 'longitude']].values
        distances = [geodesic(user_location, (lat, lon)).km for lat, lon in coords]

        # Combine predictions with operational factors
        scores = []
        for i, (prob, dist) in enumerate(zip(current_probs, distances)):
            if i == current_idx:
                continue  # Skip self

            weight = 1 / (1 + dist)
            if df.iloc[i]['site_category'] in ['CRITICAL', 'SUPER_CRITICAL']:
                weight *= 1.5
            if df.iloc[i]['bts_area'] == 'URBAN':
                weight *= 1.2
            scores.append((i, prob * weight))

        # Get top 3 recommendations
        scores.sort(key=lambda x: x[1], reverse=True)
        top_indices = [x[0] for x in scores[:3]]

        return df.iloc[top_indices][['bts_id', 'bts_name', 'latitude', 'longitude']]

    except Exception as e:
        print(f"Prediction error: {str(e)}")
        return None

In [ ]:
if __name__ == "__main__":
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]

    model, engineer, adj, df = train_model(file_name)

    if model:
        model.save('bts_handover_model.keras')
        print("Model saved successfully!")

        # Example prediction
        test_id = df.iloc[0]['bts_id']
        test_loc = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
        recommendations = predict_handover(model, engineer, adj, df, test_id, test_loc)

        if recommendations is not None:
            print("\nTop Handover Recommendations:")
            print(recommendations)
    else:
        print("Model training failed. Check error messages above.")

Saving Proj_ Dataset.xlsx to Proj_ Dataset.xlsx
Raw data loaded: 461 rows, 32 columns
Active BTS after filtering: 190

Starting training...
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0105 - loss: 4.0865 - val_accuracy: 0.0105 - val_loss: 1.1582
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.0105 - loss: 4.0794 - val_accuracy: 0.0105 - val_loss: 1.1579
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.0105 - loss: 4.0736 - val_accuracy: 0.0105 - val_loss: 1.1577
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.0105 - loss: 4.0676 - val_accuracy: 0.0105 - val_loss: 1.1574
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.0105 - loss: 4.0618 - val_accuracy: 0.0105 - val_loss: 1.1572
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.0105 - loss: 4.0528 - val_accuracy: 0.0105 - val_loss: 1.1570
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.0105 - loss: 4.0453 - val_accuracy: 